In [1]:
import os

#initialize the params
target="\\Cali\\4K\\CAM2"

#change directory for saving images|
if os.getcwd().endswith(target)==False:
    PATH=os.getcwd() + target
    os.chdir(PATH)

print(os.getcwd())

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM2


In [2]:
#with checkerboard images, do calibration
import cv2
import numpy as np
import os
import glob


#define the board's dimension
CHECKERBOARD=(7, 5)
criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#generate vector of 3D points
objpoints=[]

#generate vector of 2D points
imgpoints=[]

#define world matrix
objp=np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2]=np.mgrid[0: CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape=None

#extract the path of checkerboard images
# target='./Cali/4K/CAM1/*.jpg'

images=glob.glob(os.getcwd()+"/*.jpg")
print(images)


for fname in images:
    img=cv2.imread(fname)

    #invert into gray scale
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #find corners of the checkerboard
    ret, corners=cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    print("cv2 corner detection result: ", ret)
    
    #if the desired number of corners is found
    if ret==True:
        objpoints.append(objp)
        
        #fine tunning the matrix based on given 2D points
        corners2=cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria) 
        imgpoints.append(corners2)
        # print("corners2: ", corners2)

        #illustrate the corners
        img=cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

    elif ret==False:
        os.remove(fname)
        
    cv2.imshow('cali result', img)
    cv2.waitKey(0)



cv2.destroyAllWindows()
h, w=img.shape[:2]
print("h, w: ", h, w)

#check the results
print("objpoints: ", objpoints)

#with the true 3D points and given 2D points, do calibration
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)



['c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard0_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard10_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard11_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard12_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard13_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard14_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard15_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard16_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard17_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\checkerboard18_cam1.jpg', 'c:\\Users\\User\\Desktop\\Dev\\tabletennis_robot\\Cali\\4K\\CAM2\\che

In [3]:
print("camera matrix: \n")
print(mtx)

camera matrix: 

[[745.99261893   0.         625.02714628]
 [  0.         748.10811003 315.57402011]
 [  0.           0.           1.        ]]


In [5]:
print("dist: \n")
print(dist)

dist: 

[[ 1.52562640e-01 -4.31254941e-01 -4.27185613e-02  2.25366445e-04
  -2.04619580e-01]]


In [8]:
print("rvecs: \n")
print(rvecs)

rvecs: 

(array([[-0.02972071],
       [ 0.48175457],
       [ 3.08257579]]),)


In [9]:
print("tvecs: \n")
print(tvecs)

tvecs: 

(array([[ 2.15020573],
       [17.54851896],
       [30.14949137]]),)


In [7]:
#human error: current directory forgetting
print(os.getcwd())
target='./Cali/4K/CAM2'
images=glob.glob(target)
print(images)
for fname in images:
    print(fname)

c:\Users\User\Desktop\Dev\tabletennis_robot\Cali\4K\CAM2
[]


In [39]:
img=cv2.imread('./checkerboard0_cam1.jpg')
h, w=img.shape[:2]

#obtain newcameramtx, roi for calibration
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

In [40]:
#show the calibrated image
mapx, mapy=cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
dst=cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

x, y, w, h=roi
dst=dst[y:y+h, x:x+w]
cv2.imwrite('cali_remap0_cam1.png', dst)


True

In [16]:
#rvec, tvec for 3D mapping
import cv2
import numpy as np
import os

#define the board's dimension
CHECKERBOARD=(5, 7)
criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100000, 0.000000001)

#generate vector of 3D points
objpoints=[]

#generate vector of 2D points
imgpoints=[]

#define world matrix
objp=np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2]=np.mgrid[0: CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape=None

fname="cam1_ex.jpg"
img=cv2.imread(fname)

#invert into gray scale
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#find corners of the checkerboard
ret, corners=cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
print("cv2 corner detection result: ", ret)

#if the desired number of corners is found
if ret==True:
    objpoints.append(objp)
    
    #fine tunning the matrix based on given 2D points
    corners2=cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria) 
    imgpoints.append(corners2)
    # print("corners2: ", corners2)

    #illustrate the corners
    img=cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

# elif ret==False:
#     os.remove(fname)
    
cv2.imshow('cali result', img)
cv2.waitKey(0)



cv2.destroyAllWindows()
h, w=img.shape[:2]
print("h, w: ", h, w)

#check the results
print("objpoints: ", objpoints)

#with the true 3D points and given 2D points, do calibration
ret, mtx, dist, rvecs, tvecs=cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)



cv2 corner detection result:  True
h, w:  720 1280
objpoints:  [array([[[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [0., 1., 0.],
        [1., 1., 0.],
        [2., 1., 0.],
        [3., 1., 0.],
        [4., 1., 0.],
        [0., 2., 0.],
        [1., 2., 0.],
        [2., 2., 0.],
        [3., 2., 0.],
        [4., 2., 0.],
        [0., 3., 0.],
        [1., 3., 0.],
        [2., 3., 0.],
        [3., 3., 0.],
        [4., 3., 0.],
        [0., 4., 0.],
        [1., 4., 0.],
        [2., 4., 0.],
        [3., 4., 0.],
        [4., 4., 0.],
        [0., 5., 0.],
        [1., 5., 0.],
        [2., 5., 0.],
        [3., 5., 0.],
        [4., 5., 0.],
        [0., 6., 0.],
        [1., 6., 0.],
        [2., 6., 0.],
        [3., 6., 0.],
        [4., 6., 0.]]], dtype=float32)]
